===================================================================================================================
# DESCRIPTION
===================================================================================================================

Bismillahir Rahmaanir Raheem <br>
Almadadh Ya Gause Radi Allahu Ta'alah Anh - Ameen

by Zakia Salod <br>
Last updated: 26 October 2022

===================================================================================================================
# STEP 1
===================================================================================================================

Download (virus) FASTA file (protegen-virus-4.0-2019-01-09.faa (fasta)) from Protegen - https://violinet.org/protegen/download/index.php <br>This is the protective antigens (positive data). <br>
Output: protegen-virus-4.0-2019-01-09.fasta

===================================================================================================================
# STEP 2
===================================================================================================================

Perform similarity check of FASTA file from STEP 1. Similarity check is done using BLASTp via MMseqs2 on Ubuntu operating system. 
MMseqs2 resources: <br>
Paper: https://www.nature.com/articles/nbt.3988 <br>
Github: https://github.com/soedinglab/MMseqs2 <br>
User guide: https://mmseqs.com/latest/userguide.pdf <br>

Output: the representative sequence FASTA file - positive_mmseqs.fasta_rep_seq.fasta, which is renamed to: positive.fasta


===================================================================================================================
# NOTE: IMPORT THE FASTA FILE HERE: positive.fasta
===================================================================================================================

This file is required as input to this notebook! <br>
Especially required for STEP 5 below.

===================================================================================================================
# INSTALL REQUIRED, RELEVANT LIBRARIES
===================================================================================================================

In [ ]:
# install the Biopython library for protein sequence processing
!pip install Bio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


===================================================================================================================
# IMPORT REQUIRED, RELEVANT LIBRARIES
===================================================================================================================

In [ ]:
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import pandas as pd
from re import search
from Bio import SeqIO

In [ ]:
!python --version

Python 3.7.15


===================================================================================================================
# DECLARE GLOBAL CONSTANT VARIABLES
===================================================================================================================

In [ ]:
# CONSTANTS
DOMAIN = "https://violinet.org/protegen/"
#DOMAIN = "https://violinet.org/vaxquery/"
GRAM_POSITIVE_BACTERIA = "gram+ bacteria"
GRAM_POSITIVE_BACTERIA_START_INDEX = 1
GRAM_POSITIVE_BACTERIA_END_INDEX = 17

GRAM_NEGATIVE_BACTERIA = "gram- bacteria"
GRAM_NEGATIVE_BACTERIA_START_INDEX = 18
GRAM_NEGATIVE_BACTERIA_END_INDEX = 67

VIRUSES = "viruses"
VIRUSES_START_INDEX = 68
VIRUSES_END_INDEX = 180

PARASITES = "parasites"
PARASITES_START_INDEX = 181
PARASITES_END_INDEX = 205

FUNGI = "fungi"
FUNGI_START_INDEX = 206
FUNGI_END_INDEX = 206

OTHER_DISEASES = "other diseases"
OTHER_DISEASES_START_INDEX = 207
OTHER_DISEASES_END_INDEX = 214

===================================================================================================================
# STEP 3
===================================================================================================================

A web crawler to scrape Protegen's statistics webpage to get links for all the respective protective antigens for viruses. <br>
Protegen's statistics webpage: https://violinet.org/protegen/stat.php

In [ ]:
html = urlopen("https://violinet.org/protegen/stat.php")

In [ ]:
bsObj = BeautifulSoup(html)

In [ ]:
tables = bsObj.find("body").find_all("table")

In [ ]:
print (len(tables)) # how many tables?

2


In [ ]:
secondtable = bsObj.findAll("table")[1] # get the second table from bsObj.findAll("table") - use it 
# as a list, and just index it

In [ ]:
print(secondtable) # print the contents of the second table

<table align="center" border="0" cellpadding="3" cellspacing="1">
<tr align="center">
<td bgcolor="#AAAAAA" class="styleLeftColumn">Index </td>
<td bgcolor="#AAAAAA" class="styleLeftColumn">Pathogen Name </td>
<td bgcolor="#AAAAAA" class="styleLeftColumn">Disease</td>
<td bgcolor="#AAAAAA" class="styleLeftColumn">No. of Protective Antigens</td>
</tr>
<tr>
<td class="styleLeftColumn" colspan="4">Bacteria</td>
</tr>
<tr>
<td class="styleLeftColumn" colspan="4">G+ Bacteria</td>
</tr>
<tr>
<td align="center" bgcolor="#EEEEEE" class="styleLeftColumn">1</td>
<td bgcolor="#EEEEEE" class="smallContent"><a href="../../vaxquery/query_detail.php?c_pathogen_id=7"><i>Bacillus anthracis</i></a></td>
<td bgcolor="#EEEEEE" class="smallContent">Anthrax</td>
<td align="center" bgcolor="#EEEEEE" class="smallContent">
<a href="query_detail.php?c_pathogen_id=7">14</a>
</td>
</tr>
<tr>
<td align="center" bgcolor="#F8FAFA" class="styleLeftColumn">2</td>
<td bgcolor="#F8FAFA" class="smallContent"><a href="../

In [ ]:
def pathogen_data_per_pathogen_type(pathogen_type, start_index, end_index):
  trs = secondtable.find_all("tr") # all rows in the second table
  column_names = ["pathogen_type", "index", "pathogen_name", "link"]
  df = pd.DataFrame(columns = column_names)

  for tr in trs:
    index = tr.find("td").text
    pathogen_name = tr.find("td").text
    columns = tr.find_all('td') # all columns here
    
    if (index.isdigit()) and (start_index <= int(index) <= end_index) and (str(columns[3].contents[0].strip()) !="0"):
      link = tr.find("a") # gets the first link of each row
      df = df.append({"pathogen_type": pathogen_type, "index": index, "pathogen_name": link.text, "link":DOMAIN+link.attrs["href"]}, ignore_index=True)
  return df     

In [ ]:
get_pathogen_data = pathogen_data_per_pathogen_type(VIRUSES, VIRUSES_START_INDEX, VIRUSES_END_INDEX)

In [ ]:
# use list comprehension to remove the ../../ from the links, and put back to the link column in data frame
get_pathogen_data["link"] = [link.replace('../../vaxquery/', '') for link in get_pathogen_data["link"]]

In [ ]:
# use list comprehension to rename the pathogen_name to lower_case, and put back to the pathogen_name column in data frame
get_pathogen_data["pathogen_name"] = [name.lower() for name in get_pathogen_data["pathogen_name"]]

In [ ]:
get_pathogen_data.to_excel("step3.xlsx")

In [ ]:
get_pathogen_data.to_csv("step3.csv")

===================================================================================================================
# STEP 4
===================================================================================================================

A web crawler to scrape Protegen's protective antigens webpage by going to each link from STEP 3 above, to get the taxonomy ids data, etc, per protective antigen. <br>
An example of Protegen's protective antigens webpage is: https://violinet.org/protegen/query_detail.php?c_pathogen_id=283 for African horse sickness virus.

In [ ]:
def substring_after_delim(word, delim):
  return word.partition(delim)[2]

In [ ]:
def pathogen_data_step4(get_pathogen_data):
  column_names = ["pathogen_type", "index", "pathogen_name", "link", "taxonomy_id", "update_taxonomy_id", "protein_name", "protein_length", "protein_sequence_header", "protein_sequence"]
  df = pd.DataFrame(columns = column_names)

  for i in range(len(get_pathogen_data)):
    pathogen_type    = get_pathogen_data.loc[i, "pathogen_type"]
    index            = get_pathogen_data.loc[i, "index"]
    pathogen_name    = get_pathogen_data.loc[i, "pathogen_name"]
    link             = get_pathogen_data.loc[i, "link"]
    taxonomy_id      = ""
    update_taxonomy_id = "0"
    protein_name     = ""
    protein_length   = "" 
    protein_sequence_header = ""
    protein_sequence = ""
    print(link)
    html = urlopen(link)
    bsObj = BeautifulSoup(html)
    tables = bsObj.find("body").find_all("table")
    secondtable = bsObj.findAll("table")[1] # get the second table from bsObj.findAll("table") - use it as a list, and just index it
    trs = secondtable.find_all("tr") # all rows in the second table
    uls = secondtable.find_all("ul")
    # list comprehension to get all the lis for the above uls:
    lis = [li for ul in uls for li in ul.findAll('li')]
    for li in lis:
      if search("Taxonomy", li.text):
        taxonomy_id = substring_after_delim(li.text, ":")
        print("taxonomy_id: "+taxonomy_id)
      elif search("Protein Name", li.text):
        protein_name = substring_after_delim(li.text, ":")
        print("protein_name: "+protein_name)
      elif search("Protein Length", li.text):
        protein_length = substring_after_delim(li.text, ":")
        print("protein_length: "+protein_length)
      elif search("Protein Sequence", li.text):
        protein_sequence = substring_after_delim(li.text, "Show Sequence")
        print("protein_sequence: "+protein_sequence)
        df = df.append({"pathogen_type": pathogen_type, "index": index, "pathogen_name": pathogen_name, "link":link, "taxonomy_id":taxonomy_id, "update_taxonomy_id_in_step5":update_taxonomy_id_in_step5, "protein_name": protein_name, "protein_length":protein_length, "protein_sequence_header":protein_sequence_header, "protein_sequence": protein_sequence}, ignore_index=True)
  return df

In [ ]:
get_pathogen_data_step4 = pathogen_data_step4(get_pathogen_data)

Streaming output truncated to the last 5000 lines.


taxonomy_id:  
          11120          
protein_name:  
          S1 spike glycoprotein        
protein_length:  
          201        
protein_sequence: 

>ASV64866.1 S1 spike glycoprotein, partial [Infectious bronchitis virus]
RPSTGWHMHGGAYAVVNVSVEYRNAGSGQCTAGSIHWSKNFSASSVAMTAPVTGMSWSVSQFCTSQCNFT
NFTVFVTHCYKNGRGSCPLTGLIPQEQIRISAMKNSSLFYNLTVAVTKYPRFKSL








taxonomy_id:  
          11120          
protein_name:  
          S1 glycoprotein        
protein_length:  
          621        
protein_sequence: 

>AAQ63417.1 S1 glycoprotein, partial [Infectious bronchitis virus]
MLVKSLFLVTLLFALCSATLYDNDTYVYYYQSAFRPPVGWHLHGGAYAVVNVSQETNNAGSASQCTAGAI
HWSKNFSAASVAMTTPPSGMDWSTSQFCTAHCNFSNIVVFVTHCYKSGNNACPLTGLINQGYIRISAMKQ
GGSGPADLFYNLTVPVTKYSKFRSLQCVNNQTSVYLNGDLVFTSNETTDVVGAGVSFKAGGPITYKVMRE
VKALAYFINGTAQDVILCDRSPRGLLACQYNTGNFSDGFYPFTNSSLVKEKFIVYRENSVNTTLVLHNFT
FHNETSAPPNTVGGVDSIKVYQTQIAQSGYYNFNFSFLSGFVYKESDFMYGSYHKSCNFRPESINNGLWF
NSLTVSLAY

In [ ]:
get_pathogen_data_step4["protein_sequence_header"] = get_pathogen_data_step4["protein_sequence"].apply(lambda x : x.split(']')[0]) 

In [ ]:
# remove the > sign in the header
get_pathogen_data_step4["protein_sequence_header"] = get_pathogen_data_step4["protein_sequence_header"].str.replace(">", '')

In [ ]:
# add close brace at the end of the header
get_pathogen_data_step4["protein_sequence_header"] = get_pathogen_data_step4["protein_sequence_header"].astype(str) + ']'

In [ ]:
get_pathogen_data_step4.to_excel("step4.xlsx")

In [ ]:
get_pathogen_data_step4.to_csv("step4.csv")

===================================================================================================================
# STEP 5
===================================================================================================================

Input: positive.fasta from STEP 2 above, loop through this file using Biopython, and set taxonomy ids per record based on the file from STEP 4 above. 

In [ ]:
column_names = ["pathogen_name", "header", "cleaned_header", "taxonomy_id", "uniprot_proteome_id", "sequence"]
df_positive_data = pd.DataFrame(columns = column_names)
for seq_record in SeqIO.parse("positive.fasta", "fasta"):
    #print(seq_record.description)
    pathogen_name = substring_after_delim(seq_record.description, "[")
    pathogen_name = pathogen_name.replace(']', '')
    cleaned_header = ""
    taxonomy_id = ""
    uniprot_proteome_id = ""
    df_positive_data = df_positive_data.append({"pathogen_name": pathogen_name, "header": seq_record.description, "cleaned_header": cleaned_header, "taxonomy_id":taxonomy_id, "uniprot_proteome_id":uniprot_proteome_id, "sequence": seq_record.seq}, ignore_index=True)

In [ ]:
print(df_positive_data.count()) # there are 210 protein sequences (protective antigens)

pathogen_name     210
header            210
cleaned_header    210
taxonomy_id       210
sequence          210
dtype: int64


In [ ]:
df_positive_data.head()

,pathogen_name,header,cleaned_header,taxonomy_id,sequence
0,Zaire ebolavirus,Protegen: 18|VO: VO_0010867|NP_066249.1 minor ...,,,"(M, E, A, S, Y, E, R, G, R, P, R, A, A, R, Q, ..."
1,Zaire ebolavirus,Protegen: 19|VO: VO_0010868|NP_066245.1 matrix...,,,"(M, R, R, V, I, L, P, T, A, P, P, E, Y, M, E, ..."
2,Monkeypox virus Zaire-96-I-16,Protegen: 28|VO: VO_0010871|NP_536512.1 L1R [M...,,,"(M, D, H, N, Q, Y, L, L, T, M, F, F, A, D, D, ..."
3,Classical swine fever virus,Protegen: 442|VO: VO_0010897|AAT76713.1 E2 pro...,,,"(T, T, T, W, K, E, Y, T, H, D, L, Q, L, N, D, ..."
4,Classical swine fever virus,Protegen: 443|VO: VO_0010898|ABI94604.1 envelo...,,,"(E, N, I, T, Q, W, N, L, S, D, N, G, T, N, G, ..."


In [ ]:
print(df_positive_data["header"].head())

0    Protegen: 18|VO: VO_0010867|NP_066249.1 minor ...
1    Protegen: 19|VO: VO_0010868|NP_066245.1 matrix...
2    Protegen: 28|VO: VO_0010871|NP_536512.1 L1R [M...
3    Protegen: 442|VO: VO_0010897|AAT76713.1 E2 pro...
4    Protegen: 443|VO: VO_0010898|ABI94604.1 envelo...
Name: header, dtype: object


In [ ]:
#list comprehension to set cleaned_header as substring from the last instance of the | character 
#in the header column, until the end (needed for comparison below, for setting the taxonomy_id based on get_pathogen_data_step2)
df_positive_data["cleaned_header"] = [x.rsplit("|", 1)[-1] for x in df_positive_data["header"]]

In [ ]:
print(df_positive_data.head())

                   pathogen_name  \
0               Zaire ebolavirus   
1               Zaire ebolavirus   
2  Monkeypox virus Zaire-96-I-16   
3    Classical swine fever virus   
4    Classical swine fever virus   

                                              header  \
0  Protegen: 18|VO: VO_0010867|NP_066249.1 minor ...   
1  Protegen: 19|VO: VO_0010868|NP_066245.1 matrix...   
2  Protegen: 28|VO: VO_0010871|NP_536512.1 L1R [M...   
3  Protegen: 442|VO: VO_0010897|AAT76713.1 E2 pro...   
4  Protegen: 443|VO: VO_0010898|ABI94604.1 envelo...   

                                      cleaned_header taxonomy_id  \
0  NP_066249.1 minor nucleoprotein [Zaire ebolavi...               
1      NP_066245.1 matrix protein [Zaire ebolavirus]               
2    NP_536512.1 L1R [Monkeypox virus Zaire-96-I-16]               
3  AAT76713.1 E2 protein, partial [Classical swin...               
4  ABI94604.1 envelope glycoprotein E0, partial [...               

                                     

In [ ]:
get_pathogen_data_step4.head()

,pathogen_type,index,pathogen_name,link,taxonomy_id,update_taxonomy_in_positive_data,protein_name,protein_length,protein_sequence_header,protein_sequence
0,viruses,69,african horse sickness virus,https://violinet.org/protegen/query_detail.php...,\r\n 36421,0,\r\n NS1,\r\n 606,\n\nAKP39984.1 NS1 [African horse sickness vir...,\n\n>AKP39984.1 NS1 [African horse sickness vi...
1,viruses,69,african horse sickness virus,https://violinet.org/protegen/query_detail.php...,\r\n 40050,0,\r\n VP2,\r\n 141,"\n\nCDJ98824.1 VP2, partial [African horse sic...","\n\n>CDJ98824.1 VP2, partial [African horse si..."
2,viruses,69,african horse sickness virus,https://violinet.org/protegen/query_detail.php...,\r\n 36421,0,\r\n VP7,\r\n 404,\n\nAKP39983.1 VP7 [African horse sickness vir...,\n\n>AKP39983.1 VP7 [African horse sickness vi...
3,viruses,70,african swine fever virus,https://violinet.org/protegen/query_detail.php...,\r\n 10497,0,\r\n CD2v,\r\n 229,"\n\nAJB28435.1 CD2v, partial [African swine fe...","\n\n>AJB28435.1 CD2v, partial [African swine f..."
4,viruses,70,african swine fever virus,https://violinet.org/protegen/query_detail.php...,\r\n 10497,0,\r\n p32,\r\n 233,\n\nAGL93215.1 p32 [African swine fever virus],\n\n>AGL93215.1 p32 [African swine fever virus...


In [ ]:
#set the taxonomy ids in df_positive_data
#update the update_taxonomy_id to be 1 (yes) in get_pathogen_data_step2 if the taxonomy id is updated in df_positive_data
count=0
for i, row1 in get_pathogen_data_step4.iterrows():
  #print("Index", i, "- Column protein_sequence_header:", row1["protein_sequence_header"])
  for j, row2 in df_positive_data.iterrows():
    #if row2["cleaned_header"].str.strip().equals(row1["protein_sequence_header"].str.strip()):
    if row2["cleaned_header"].strip() == row1["protein_sequence_header"].strip():
      df_positive_data["taxonomy_id"].values[j] = row1["taxonomy_id"]
      get_pathogen_data_step4["update_taxonomy_id"].values[i] = "1"
      df_positive_data["pathogen_name"].values[j] = get_pathogen_data_step4["pathogen_name"].values[i]
      count+=1
print("count = "+str(count))

In [ ]:
df_positive_data.to_excel("step5.xlsx") # this is renamed to: Supplementary Table S2.xlsx

In [ ]:
# re-export this step4.xlsx file (overwrite) from step 4, since the update_taxonomy_id field 
# of the get_pathogen_data_step4 data frame was updated, used to keep track of which protective sequences' 
# taxonomy_id were updated in step5.xlsx
get_pathogen_data_step4.to_excel("step4.xlsx") # this is renamed to: Supplementary Table S1.xlsx

In [ ]:
# re-export this step4.csv (overwrite) file from step 4, since the update_taxonomy_id field 
# of the get_pathogen_data_step4 data frame was updated, used to keep track of which protective sequences' 
# taxonomy_id were updated in step5.xlsx
get_pathogen_data_step4.to_csv("step4.csv")